In [44]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# 1. Loading in the data

In [45]:
fema = pd.read_csv('../../data/DisasterDeclarationsSummaries.csv')

In [46]:
hurricanes = pd.read_csv('../../data/hurdat2_flat.csv')

In [47]:
migration_folder = '../../data/IRS Migration'

year_mappings = {
    '1617': {'year':2017, 'label':'2016-2017'},
    '1718': {'year':2018, 'label':'2017-2018'},
    '1819': {'year':2019, 'label':'2018-2019'},
    '1920': {'year':2020, 'label':'2019-2020'},
    '2021': {'year':2021, 'label':'2020-2021'},
    '2122': {'year':2022, 'label':'2021-2022'}
}


In [48]:
all_inflow = []
all_outflow = []

for year_code, year_info in year_mappings.items():
    inflow_file = os.path.join(migration_folder, f'countyinflow{year_code}.csv')
    outflow_file = os.path.join(migration_folder, f'countyoutflow{year_code}.csv')
    
    inflow_df = pd.read_csv(inflow_file, encoding='cp1252')
    outflow_df = pd.read_csv(outflow_file, encoding='cp1252')
    
    inflow_df['year'] = year_info['year']
    inflow_df['year_label'] = year_info['label']
    
    outflow_df['year'] = year_info['year']
    outflow_df['year_label'] = year_info['label']
    
    all_inflow.append(inflow_df)
    all_outflow.append(outflow_df)

In [49]:
inflow_combined = pd.concat(all_inflow, ignore_index=True)
outflow_combined = pd.concat(all_outflow, ignore_index=True)

In [50]:
len(inflow_combined), len(outflow_combined)

sorted(inflow_combined['year'].unique()), sorted(outflow_combined['year'].unique())

([np.int64(2017),
  np.int64(2018),
  np.int64(2019),
  np.int64(2020),
  np.int64(2021),
  np.int64(2022)],
 [np.int64(2017),
  np.int64(2018),
  np.int64(2019),
  np.int64(2020),
  np.int64(2021),
  np.int64(2022)])

In [51]:
# Filter to only Florida (state FIPS code = 12)

inflow_combined = inflow_combined[
    (inflow_combined['y2_statefips'] == 12)
].copy()

outflow_combined = outflow_combined[
    (outflow_combined['y1_statefips'] == 12)
].copy()

# 2. Preparing FEMA Data

In [52]:
# Only FL
fema = fema[fema['state'] == 'FL'].copy()
print(f"Filtered FEMA to FL: {len(fema)} rows")

Filtered FEMA to FL: 2791 rows


In [53]:
# Re-classing dates

fema['declarationDate'] = pd.to_datetime(fema['declarationDate'])
fema['incidentBeginDate'] = pd.to_datetime(fema['incidentBeginDate'])
fema['incidentEndDate'] = pd.to_datetime(fema['incidentEndDate'])

In [54]:
# Creating combined FIPS

fema['fips'] = (fema['fipsStateCode'].astype(str).str.zfill(2) + 
fema['fipsCountyCode'].astype(str).str.zfill(3))

# Adding year for matching

fema['year'] = fema['incidentBeginDate'].dt.year

In [55]:
# Quick validation

print(f"Data range: {fema['incidentBeginDate'].min()} - {fema['incidentBeginDate'].max()}")

Data range: 1953-10-22 00:00:00+00:00 - 2024-10-05 00:00:00+00:00


In [56]:
# Isolating hurricane-related declarations

hurricane_keywords = ['hurricane', 'tropical storm', 'tropical', 'cyclone', 'severe storm']
hurricane_mask = (fema['incidentType'].str.lower().str.contains('|'.join(hurricane_keywords), na=False) | 
fema['declarationTitle'].str.lower().str.contains('|'.join(hurricane_keywords), na=False))

fema_hurricanes = fema[hurricane_mask].copy()

In [57]:
# Filtering to 2017-2022 only

fema_hurricanes = fema_hurricanes[
    (fema_hurricanes['year'] >= 2017) & 
    (fema_hurricanes['year'] <= 2022)
].copy()

In [58]:
# Quick validation

print(f"There are {len(fema_hurricanes)} hurricane-related disaster declarations, spanning {fema_hurricanes['year'].min()} to {fema_hurricanes['year'].max()}.")
print(f"Years that have hurricane declarations: {sorted(fema_hurricanes['year'].unique())}")

There are 663 hurricane-related disaster declarations, spanning 2017 to 2022.
Years that have hurricane declarations: [np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022)]


# 3. Preparing IRS Migration Data

In [59]:
# FIPS Codes for inflow

inflow_combined['destination_fips'] = (inflow_combined['y2_statefips'].astype(str).str.zfill(2) + 
inflow_combined['y2_countyfips'].astype(str).str.zfill(3))

inflow_combined['origin_fips'] = (inflow_combined['y1_statefips'].astype(str).str.zfill(2) + 
inflow_combined['y1_countyfips'].astype(str).str.zfill(3))

In [60]:
# FIPS Codes for outflow

outflow_combined['origin_fips'] = (outflow_combined['y1_statefips'].astype(str).str.zfill(2) + 
outflow_combined['y1_countyfips'].astype(str).str.zfill(3))

outflow_combined['destination_fips'] = (outflow_combined['y2_statefips'].astype(str).str.zfill(2) + 
outflow_combined['y2_countyfips'].astype(str).str.zfill(3))

In [61]:
# Aggregating by county and year

outflow_summary = outflow_combined.groupby(['origin_fips', 'year']).agg({
    'n1' : 'sum', # Tax Returns (Households)
    'n2' : 'sum', # People
    'agi' : 'sum' # Income
}).reset_index()

outflow_summary.columns = ['fips', 'year', 'outflow_returns', 'outflow_people', 'outflow_agi']

inflow_summary = inflow_combined.groupby(['destination_fips', 'year']).agg({
    'n1' : 'sum', # Tax Returns (Households)
    'n2' : 'sum', # People
    'agi' : 'sum' # Income
}).reset_index()

inflow_summary.columns = ['fips', 'year', 'inflow_returns', 'inflow_people', 'inflow_agi']

In [62]:
# Merging

migration_summary = outflow_summary.merge(
    inflow_summary,
    on=['fips', 'year'],
    how='outer'
)

In [63]:
# Adding columns for net migration

migration_summary['net_migration_returns'] = (
    migration_summary['inflow_returns'] - migration_summary['outflow_returns']
)

migration_summary['net_migration_people'] = (
    migration_summary['inflow_people'] - migration_summary['outflow_people']
)

In [64]:
print(f"Unique county-year combinations: {len(migration_summary):,}")

Unique county-year combinations: 404


# 4. Preparing HURDAT Data

In [65]:
hurricanes = pd.read_csv('../../data/hurdat2_flat.csv')

In [66]:
hurricanes.head()

,Storm_ID,Storm_Name,Date,Time,Status,Latitude,Longitude,Latitude_Numeric,Longitude_Numeric,Max_Wind,...,NW_34kt,NE_50kt,SE_50kt,SW_50kt,NW_50kt,NE_64kt,SE_64kt,SW_64kt,NW_64kt,Max_Wind_Radius
0,AL011851,UNNAMED,1851-06-25,0,HU,28.0N,94.8W,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL011851,UNNAMED,1851-06-25,600,HU,28.0N,95.4W,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL011851,UNNAMED,1851-06-25,1200,HU,28.0N,96.0W,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AL011851,UNNAMED,1851-06-25,1800,HU,28.1N,96.5W,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AL011851,UNNAMED,1851-06-25,2100,HU,28.2N,96.8W,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
hurricanes['Date'] = pd.to_datetime(hurricanes['Date'], format='%Y-%m-%d', errors='coerce')
hurricanes['Year'] = hurricanes['Date'].dt.year

In [68]:
# Filtering so it matches IRS

available_years = sorted(migration_summary['year'].unique())
hurricanes_filtered = hurricanes[hurricanes['Year'].isin(available_years)].copy()

storm_summary = hurricanes_filtered.groupby(['Storm_ID']).agg({
    'Storm_Name': 'first',
    'Date': ['min', 'max'],
    'Max_Wind': 'max',
    'Min_Pressure': 'min',
    'Year': 'first'
}).reset_index()

storm_summary.columns = ['Storm_ID', 'Storm_Name', 'Start_Date', 'End_Date', 
                         'Max_Wind_Speed', 'Min_Pressure', 'Year']

In [69]:
print(f"  Years: {available_years}")

  Years: [np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022)]


In [70]:
storm_summary.head()

,Storm_ID,Storm_Name,Start_Date,End_Date,Max_Wind_Speed,Min_Pressure,Year
0,AL012017,ARLENE,2017-04-16,2017-04-22,55,986.0,2017
1,AL012018,ALBERTO,2018-05-25,2018-05-31,55,990.0,2018
2,AL012019,ANDREA,2019-05-20,2019-05-22,35,1006.0,2019
3,AL012020,ARTHUR,2020-05-16,2020-05-21,55,989.0,2020
4,AL012021,ANA,2021-05-20,2021-05-24,50,1004.0,2021


In [71]:
print(f"In the available years for migration data, there are {len(storm_summary)} unique storms.")
print(f"The storms start as early as {storm_summary['Start_Date'].min().date()} and go until {storm_summary['End_Date'].max().date()}.")

In the available years for migration data, there are 122 unique storms.
The storms start as early as 2017-04-16 and go until 2022-11-11.


# 5. Merging

In [72]:
# Merging Declarations with IRS Migration

fema_migration = fema_hurricanes.merge(
    migration_summary,
    on=['fips', 'year'],
    how='left'
)

In [73]:
fema_migration.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,fips,year,outflow_returns,outflow_people,outflow_agi,inflow_returns,inflow_people,inflow_agi,net_migration_returns,net_migration_people
0,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,12001,2022,133864.0,236905.0,10594863.0,131143.0,235518.0,10528909.0,-2721.0,-1387.0
1,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,12003,2022,12078.0,25438.0,747938.0,12620.0,26869.0,806790.0,542.0,1431.0
2,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,12007,2022,10888.0,21990.0,601863.0,12262.0,24667.0,666370.0,1374.0,2677.0
3,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,12011,2022,999987.0,1808085.0,93860199.0,994604.0,1786372.0,99337673.0,-5383.0,-21713.0
4,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,12013,2022,4986.0,10408.0,261886.0,5402.0,11180.0,286455.0,416.0,772.0


In [74]:
# Adding HURDAT2 Storm Info

fema_migration_storms = fema_migration.copy()

if len(storm_summary) > 0:
    for idx, row in storm_summary.iterrows():
        storm_name = row['Storm_Name']
        if pd.notna(storm_name):
            mask = (
                fema_migration_storms['declarationTitle'].str.contains(storm_name, case=False, na=False) &
                (fema_migration_storms['year'] == row['Year'])
            )
            fema_migration_storms.loc[mask, 'matched_storm_id'] = row['Storm_ID']
            fema_migration_storms.loc[mask, 'max_wind_speed'] = row['Max_Wind_Speed']
            fema_migration_storms.loc[mask, 'min_pressure'] = row['Min_Pressure']

    matched = fema_migration_storms['matched_storm_id'].notna().sum()
    print(f"Matched {matched} records to storm tracks.")

Matched 663 records to storm tracks.


In [75]:
fema_migration_storms.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,outflow_people,outflow_agi,inflow_returns,inflow_people,inflow_agi,net_migration_returns,net_migration_people,matched_storm_id,max_wind_speed,min_pressure
0,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,236905.0,10594863.0,131143.0,235518.0,10528909.0,-2721.0,-1387.0,AL172022,65.0,980.0
1,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,25438.0,747938.0,12620.0,26869.0,806790.0,542.0,1431.0,AL172022,65.0,980.0
2,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,21990.0,601863.0,12262.0,24667.0,666370.0,1374.0,2677.0,AL172022,65.0,980.0
3,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,1808085.0,93860199.0,994604.0,1786372.0,99337673.0,-5383.0,-21713.0,AL172022,65.0,980.0
4,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,10408.0,261886.0,5402.0,11180.0,286455.0,416.0,772.0,AL172022,65.0,980.0


# 6. Creating Summaries

## 6.1. Hurricane impact by county

In [76]:
# Hurricane impact by county

hurricane_county_summary = fema_migration_storms.groupby('fips').agg({
    'disasterNumber': 'count',
    'state': 'first',
    'designatedArea': 'first',
    'year': lambda x: ', '.join(map(str, sorted(x.unique()))),
    'outflow_returns': 'sum',
    'inflow_returns': 'sum',
    'net_migration_returns': 'sum',
    'outflow_people': 'sum',
    'inflow_people': 'sum',
    'net_migration_people': 'sum',
    'outflow_agi': 'sum', # Income leaving the county
    'inflow_agi': 'sum', # Income entering the county
    'max_wind_speed': 'max'
}).reset_index()

# Renaming for aesthetic reasons

hurricane_county_summary.columns = [
        'fips', 'num_hurricane_declarations', 'state', 'county_name', 'years_affected',
        'outflow_returns', 'inflow_returns', 'net_migration_returns',
        'outflow_people', 'inflow_people', 'net_migration_people',
        'outflow_agi', 'inflow_agi', 'max_wind_speed'
    ]

In [77]:
hurricane_county_summary.head()

,fips,num_hurricane_declarations,state,county_name,years_affected,outflow_returns,inflow_returns,net_migration_returns,outflow_people,inflow_people,net_migration_people,outflow_agi,inflow_agi,max_wind_speed
0,12000,34,FL,Big Cypress Indian Reservation,"2017, 2019, 2022",14521283.0,16434607.0,1913324.0,26537287.0,30565927.0,4028640.0,798212478.0,1.124438e+09,160.0
1,12001,10,FL,Alachua (County),"2017, 2018, 2019, 2020, 2021, 2022",1298659.0,1258165.0,-40494.0,2359026.0,2320187.0,-38839.0,93275814.0,9.178640e+07,160.0
2,12003,8,FL,Baker (County),"2017, 2018, 2019, 2021, 2022",89801.0,94208.0,4407.0,194985.0,206382.0,11397.0,5207214.0,5.589318e+06,160.0
3,12005,10,FL,Bay (County),"2017, 2018, 2019, 2020, 2021, 2022",924011.0,931916.0,7905.0,1818525.0,1833234.0,14709.0,57269254.0,6.022466e+07,160.0
4,12007,9,FL,Bradford (County),"2017, 2018, 2019, 2021, 2022",96450.0,105055.0,8605.0,198636.0,215506.0,16870.0,4968764.0,5.359301e+06,160.0


In [78]:
# Creating a new column indicating rate of migration

hurricane_county_summary['outflow_rate'] = (
    hurricane_county_summary['outflow_returns'] / 
    (hurricane_county_summary['outflow_returns'] + hurricane_county_summary['inflow_returns'] * 100)
)

hurricane_county_summary['outflow_rate'].head()

0    0.008758
1    0.010216
2    0.009442
3    0.009818
4    0.009097
Name: outflow_rate, dtype: float64

## 6.2. County-year level

In [79]:
with pd.option_context('display.max_columns', None):
    display(fema_migration_storms.head())

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,disasterCloseoutDate,tribalRequest,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id,fips,year,outflow_returns,outflow_people,outflow_agi,inflow_returns,inflow_people,inflow_agi,net_migration_returns,net_migration_people,matched_storm_id,max_wind_speed,min_pressure
0,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,1,1,2022-11-07 00:00:00+00:00,2022-11-30 00:00:00+00:00,NaN,0,12,1,99001,Alachua (County),22104,2023-02-13T00:00:00.000Z,2022110701,4,"4,H",2024-08-27T18:22:14.800Z,93a59d4d8f26ec5507f34aeb06b86fe1e98f2f3d,c2833057-166d-4f85-a53f-5e4ae70b1b5e,12001,2022,133864.0,236905.0,10594863.0,131143.0,235518.0,10528909.0,-2721.0,-1387.0,AL172022,65.0,980.0
1,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,1,1,2022-11-07 00:00:00+00:00,2022-11-30 00:00:00+00:00,NaN,0,12,3,99003,Baker (County),22104,2023-02-13T00:00:00.000Z,2022110701,4,"4,H",2024-08-27T18:22:14.800Z,74125142f6e1e0dc18bc4c90e832e6b3f32bff13,6bc776fc-98e1-40f4-ab8d-c393a7cf0326,12003,2022,12078.0,25438.0,747938.0,12620.0,26869.0,806790.0,542.0,1431.0,AL172022,65.0,980.0
2,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,1,1,2022-11-07 00:00:00+00:00,2022-11-30 00:00:00+00:00,NaN,0,12,7,99007,Bradford (County),22104,2023-02-13T00:00:00.000Z,2022110701,4,"4,H",2024-08-27T18:22:14.800Z,55ca100e2799d9613ce8dd9395ca5acc679de40c,5a5896ee-efca-45f4-85de-4cdc55c4f2c5,12007,2022,10888.0,21990.0,601863.0,12262.0,24667.0,666370.0,1374.0,2677.0,AL172022,65.0,980.0
3,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,1,1,2022-11-07 00:00:00+00:00,2022-11-30 00:00:00+00:00,NaN,0,12,11,99011,Broward (County),22104,2023-02-13T00:00:00.000Z,2022110701,4,"4,H",2024-08-27T18:22:14.800Z,e8fb0634984472c53d452fe862f458853a73ba43,202f3a2d-9bc2-47eb-917a-723b4cf7f588,12011,2022,999987.0,1808085.0,93860199.0,994604.0,1786372.0,99337673.0,-5383.0,-21713.0,AL172022,65.0,980.0
4,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,1,1,2022-11-07 00:00:00+00:00,2022-11-30 00:00:00+00:00,NaN,0,12,13,99013,Calhoun (County),22104,2023-02-13T00:00:00.000Z,2022110701,4,"4,H",2024-08-27T18:22:14.800Z,bede4bd4b03e51fd25cb1d73d958d6b0d3877299,2cf3a8c1-765c-4309-9535-672f6de1d314,12013,2022,4986.0,10408.0,261886.0,5402.0,11180.0,286455.0,416.0,772.0,AL172022,65.0,980.0


In [80]:
hurricane_county_year = fema_migration_storms.groupby(['fips', 'year']).agg({
    'disasterNumber': 'count',
    'state': 'first',
    'designatedArea': 'first',
    'outflow_returns': 'first',
    'inflow_returns': 'first',
    'net_migration_returns': 'first',
    'outflow_people': 'first',
    'inflow_people': 'first',
    'net_migration_people': 'first',
    'outflow_agi': 'first',
    'inflow_agi': 'first',
    'max_wind_speed': 'max',
    'declarationTitle': lambda x: '; '.join(x.unique())
}).reset_index()

# Renaming for aesthetic reasons

hurricane_county_year.columns = [
    'fips', 'year', 'num_declarations', 'state', 'county_name',
    'outflow_returns', 'inflow_returns', 'net_migration_returns',
    'outflow_people', 'inflow_people', 'net_migration_people',
    'outflow_agi', 'inflow_agi', 'max_wind_speed', 'storms'
]

In [82]:
hurricane_county_year.isnull().sum()

fips                     0
year                     0
num_declarations         0
state                    0
county_name              0
outflow_returns          2
inflow_returns           2
net_migration_returns    2
outflow_people           2
inflow_people            2
net_migration_people     2
outflow_agi              2
inflow_agi               2
max_wind_speed           0
storms                   0
dtype: int64

In [86]:
with pd.option_context('display.max_columns', None):
    display(hurricane_county_year.head())

,fips,year,num_declarations,state,county_name,outflow_returns,inflow_returns,net_migration_returns,outflow_people,inflow_people,net_migration_people,outflow_agi,inflow_agi,max_wind_speed,storms
0,12000,2017,7,FL,Big Cypress Indian Reservation,2074469.0,2347801.0,273332.0,3791041.0,4366561.0,575520.0,114030354.0,160634007.0,155.0,HURRICANE IRMA - SEMINOLE TRIBE OF FLORIDA; HU...
1,12000,2019,6,FL,Big Cypress Indian Reservation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,HURRICANE DORIAN
2,12000,2022,21,FL,Big Cypress Indian Reservation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,HURRICANE IAN - SEMINOLE TRIBE OF FLORIDA; HUR...
3,12001,2017,2,FL,Alachua (County),129077.0,123305.0,-5772.0,240932.0,233654.0,-7278.0,7964561.0,7645654.0,155.0,HURRICANE IRMA
4,12001,2018,1,FL,Alachua (County),123477.0,119271.0,-4206.0,229029.0,224269.0,-4760.0,7863874.0,7712832.0,140.0,HURRICANE MICHAEL


## 6.3. Storm-level analysis

In [87]:
storm_impact = fema_migration_storms.groupby(['declarationTitle', 'year']).agg({
    'fips': 'nunique',
    'state': lambda x: ', '.join(x.unique()),
    'incidentBeginDate': 'first',
    'incidentEndDate': 'first',
    'outflow_returns': 'sum',
    'inflow_returns': 'sum',
    'net_migration_returns': 'sum',
    'outflow_people': 'sum',
    'inflow_people': 'sum',
    'net_migration_people': 'sum',
    'max_wind_speed': 'max'
}).reset_index()

storm_impact.columns = [
    'storm_name', 'year', 'counties_affected', 'states', 'start_date', 'end_date',
    'total_outflow_returns', 'total_inflow_returns', 'net_migration_returns',
    'total_outflow_people', 'total_inflow_people', 'net_migration_people',
    'max_wind_speed'
]

In [88]:
storm_impact.head()

,storm_name,year,counties_affected,states,start_date,end_date,total_outflow_returns,total_inflow_returns,net_migration_returns,total_outflow_people,total_inflow_people,net_migration_people,max_wind_speed
0,HURRICANE DORIAN,2019,14,FL,2019-08-28 00:00:00+00:00,2019-09-09 00:00:00+00:00,3698332.0,3804258.0,105926.0,7124149.0,7339404.0,215255.0,160.0
1,HURRICANE DORIAN,2019,68,FL,2019-08-28 00:00:00+00:00,2019-09-09 00:00:00+00:00,10288218.0,10557352.0,269134.0,19809584.0,20358220.0,548636.0,160.0
2,HURRICANE ETA,2020,13,FL,2020-11-07 00:00:00+00:00,2020-11-12 00:00:00+00:00,2454637.0,2614847.0,160210.0,4711829.0,5026366.0,314537.0,130.0
3,HURRICANE IAN,2022,68,FL,2022-09-23 00:00:00+00:00,2022-11-04 00:00:00+00:00,11157556.0,11679277.0,521721.0,20470591.0,21500369.0,1029778.0,140.0
4,HURRICANE IAN - SEMINOLE TRIBE OF FLORIDA,2022,1,FL,2022-09-23 00:00:00+00:00,2022-11-04 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,140.0


In [89]:
storm_impact = storm_impact.sort_values(['year', 'net_migration_people']).reset_index(drop=True)
storm_impact.head()

,storm_name,year,counties_affected,states,start_date,end_date,total_outflow_returns,total_inflow_returns,net_migration_returns,total_outflow_people,total_inflow_people,net_migration_people,max_wind_speed
0,HURRICANE NATE,2017,2,FL,2017-10-07 00:00:00+00:00,2017-10-11 00:00:00+00:00,244205.0,255926.0,11721.0,496954.0,522966.0,26012.0,80.0
1,HURRICANE IRMA,2017,68,FL,2017-09-04 00:00:00+00:00,2017-10-18 00:00:00+00:00,22355369.0,23395933.0,1040564.0,43232341.0,45432841.0,2200500.0,155.0
2,HURRICANE IRMA - SEMINOLE TRIBE OF FLORIDA,2017,1,FL,2017-09-04 00:00:00+00:00,2017-10-04 00:00:00+00:00,12446814.0,14086806.0,1639992.0,22746246.0,26199366.0,3453120.0,155.0
3,HURRICANE MICHAEL,2018,35,FL,2018-10-07 00:00:00+00:00,2018-10-19 00:00:00+00:00,3178448.0,3283956.0,105508.0,6218154.0,6438886.0,220732.0,140.0
4,HURRICANE DORIAN,2019,14,FL,2019-08-28 00:00:00+00:00,2019-09-09 00:00:00+00:00,3698332.0,3804258.0,105926.0,7124149.0,7339404.0,215255.0,160.0


## 6.4. Trends over time

In [90]:
migration_trends = migration_summary.groupby('year').agg({
    'fips': 'count',
    'outflow_returns': 'sum',
    'inflow_returns': 'sum',
    'net_migration_returns': 'sum',
    'outflow_people': 'sum',
    'inflow_people': 'sum',
    'net_migration_people': 'sum'
}).reset_index()

migration_trends.columns = [
    'year', 'counties_with_data', 'total_outflow_returns', 'total_inflow_returns',
    'net_migration_returns', 'total_outflow_people', 'total_inflow_people', 'net_migration_people'
]

In [91]:
migration_trends.head()

,year,counties_with_data,total_outflow_returns,total_inflow_returns,net_migration_returns,total_outflow_people,total_inflow_people,net_migration_people
0,2017,68,12214919,12871867,656948,23511691,24899701,1388010
1,2018,68,11767323,12183814,416491,22504353,23372328,867975
2,2019,67,10288218,10557352,269134,19809584,20358220,548636
3,2020,67,10512010,10855127,343117,20435194,21146454,711260
4,2021,67,10824102,11361031,536929,20060013,21142945,1082932


# 7. Exporting

In [92]:
os.makedirs('data/merge', exist_ok=True)

In [93]:
fema_migration_storms.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,outflow_people,outflow_agi,inflow_returns,inflow_people,inflow_agi,net_migration_returns,net_migration_people,matched_storm_id,max_wind_speed,min_pressure
0,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,236905.0,10594863.0,131143.0,235518.0,10528909.0,-2721.0,-1387.0,AL172022,65.0,980.0
1,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,25438.0,747938.0,12620.0,26869.0,806790.0,542.0,1431.0,AL172022,65.0,980.0
2,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,21990.0,601863.0,12262.0,24667.0,666370.0,1374.0,2677.0,AL172022,65.0,980.0
3,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,1808085.0,93860199.0,994604.0,1786372.0,99337673.0,-5383.0,-21713.0,AL172022,65.0,980.0
4,DR-4680-FL,4680,FL,DR,2022-12-13 00:00:00+00:00,2023,Hurricane,HURRICANE NICOLE,0,0,...,10408.0,261886.0,5402.0,11180.0,286455.0,416.0,772.0,AL172022,65.0,980.0


In [94]:
# 1. Merged dataset

output_file = os.path.join('data/merge', 'hurricane_migration_combined.csv')
fema_migration_storms.to_csv(output_file, index=False)

In [95]:
# 2. County summary

output_file = os.path.join('data/merge', 'hurricane_county_impacts.csv')
hurricane_county_summary.to_csv(output_file, index=False)

In [96]:
# 3. County-year summary

output_file = os.path.join('data/merge', 'hurricane_county_year_impacts.csv')
hurricane_county_year.to_csv(output_file, index=False)

In [97]:
# 4. Storm impact summary

output_file = os.path.join('data/merge', 'hurricane_storm_impacts.csv')
storm_impact.to_csv(output_file, index=False)